In [1]:
import numpy as np
import pandas as pd
from rapidfuzz import fuzz

In [2]:
#read booking and trivago item metadata files
booking = pd.read_csv('final_booking.csv')
trivago = pd.read_csv('trivago_item_metadata.csv')

In [ ]:
#Comparison of the features of each hotel in the booking table with the trivago hotel features to find similarities 
#keep the index of the hotel with the highest similarity

array = []
lines = []
index = []
copyTrivago = trivago.copy()

for j in range(len(booking)): 
    for i in range(len(trivago)):
        if i not in index:
            line = copyTrivago['properties'][i]
            array.append(tuple([i,fuzz.token_sort_ratio(booking['property'][j], line)])) #i-1 yap

    maximum =  max(array,key=lambda item:item[1])[0]
    similarity = max(array,key=lambda item:item[1])[1]
    print("line:", j,"\tmaximum:", maximum,"\tsimilarity:", similarity)
    lines.append(tuple([j,maximum,similarity]))
    index.append(maximum)
    array = []
    
#create new similarities csv (include index of booking, index of trivago and similarity ratio)
similarity_table = {
    "line":[i[0] for i in lines] ,
    "maximum":[i[1] for i in lines],
    "similarity":[i[2] for i in lines]
}
similarities = pd.DataFrame(similarity_table)
similarities.to_csv("similarities.csv", index=False)

In [3]:
#read new similarities file
similarities = pd.read_csv("similarities.csv")
trivago = pd.read_csv('trivago_item_metadata.csv')
booking = pd.read_csv('final_booking.csv')

In [8]:
#find the id of the empty rows to be deleted
print(similarities['line'][50434])
print(similarities['line'][50435])

50434
52754


In [9]:
#find the id of the empty rows to be deleted
print(similarities['line'][100869])
print(similarities['line'][100870])

103188
105508


In [10]:
#find the id of the empty rows to be deleted
print(similarities['line'][151305])

155943


In [13]:
#delete from the booking table according to the found ids 
booking.drop(booking.loc[50435:52753].index, inplace=True)
booking.drop(booking.loc[103189:105507].index, inplace=True)
booking.drop(booking.loc[155944:].index, inplace=True)

In [ ]:
#match the ids of 2 tables
for i in range(len(similarities)):
    booking["id"][similarities["line"][i]] = trivago["item_id"][similarities["maximum"][i]]
    
#create new item metadata csv    
booking.to_csv("final_item_metadata.csv", index=False)

#read new final_item_metadata file
booking = pd.read_csv("final_item_metadata.csv")